In [1]:
import os
import sys
os.chdir('..')
current_dir = os.path.abspath('')
sys.path.append(current_dir)

In [5]:
import scripts.helpers
import scripts.sampler_helpers
import scripts.latent_helpers
from pytorch_lightning import seed_everything
import importlib

In [3]:
importlib.reload(scripts.helpers)
importlib.reload(scripts.sampler_helpers)
importlib.reload(scripts.latent_helpers)
from scripts.helpers import *
from scripts.sampler_helpers import *
from scripts.latent_helpers import *

In [4]:
seed_everything(42)
SAVE_PATH = "outputs/latentimgs_test/"
set_lowvram_mode(False)
model = load_model_from_joblib("./checkpoints/sdxlbase1_cache.joblib")

Global seed set to 42


In [6]:
options = {
  "discretization": "EDMDiscretization",
  "sigma_min"     : 0.03,   #EDMDiscretization, [-inf, inf | 0.03]
  "sigma_max"     : 14.61,  #EDMDiscretization, [-inf, inf | 14.61]
  "rho"           : 3.0,    #EDMDiscretization, [-inf, inf | 3.0]

  "guider"                  : "VanillaCFG",
  "additional_guider_kwargs": {},
  "vanilla_cfg"             : 5.0,  #VanillaCFG, [0.0, inf | 5.0]
  "linear_cfg"              : 1.5,  #LinearCFG, [1.0, inf | 1.5]
  "triangle_cfg"            : 2.5,  #TriangleCFG, [1.0, 10.0 | 2.5]
  "min_cfg"                 : 1.0,  #LinearCFG TriangleCFG, [1.0]
  "num_frames"              : 25,   #LinearCFG TriangleCFG, [25]

  "sampler"           : "DPMPP2MSampler",
  "s_churn"           : 0.0,    #EulerEDM HeunEDM [0.0, inf | 0.0]
  "s_tmin"            : 0.0,    #EulerEDM HeunEDM [0.0, inf |0.0]
  "s_tmax"            : 999.0,  #EulerEDM HeunEDM [0.0, inf | 999.0]
  "s_noise"           : 0.0,    #EulerEDM HeunEDM [0.0, inf | 0.0]
  "eta"               : 1.0,    #EulerAncestral DPMPP2SAncestral [0.0, inf | 1.0]
  "s_noise_ancestral" : 1.0,    #EulerAncestral DPMPP2SAncestral [0.0, inf | 1.0]
  "order"             : 4,      #LinearMultistep [1, inf | 4]

  "crop_coords_top"         : 0,    #[0, inf | 0]
  "crop_coords_left"        : 0,    #[0, inf | 0]
  "aesthetic_score"         : 6.0,  #[-inf, inf | 6.0]
  "negative_aesthetic_score": 2.5,  #[-inf, inf | 2.5]
  "fps"                     : 6,    #[1, inf | 6]
  "mb_id"                   : 127,  #[0, 511 | 127]
  "image_path"              : None
}

In [35]:
prompt = "A cow happily floating in outer space"
negative_prompt = ""
num_samples =  1
num_steps =  30
height = 576
width = 1024

sampler = init_sampling(options = options, steps = num_steps)
value_dict = {
    'crop_coords_top': 0,
    'crop_coords_left': 0,
    'orig_width': width,
    'orig_height': height,
    'target_width': width,
    'target_height': height,
    'prompt': prompt,
    'negative_prompt': negative_prompt
}

In [16]:
c, uc = get_conditionings(model, value_dict, num_samples)

In [37]:
samples = get_samples(model, sampler, num_samples, c, uc, height, width)

##############################  Sampling setting  ##############################
Sampler: DPMPP2MSampler
Discretization: EDMDiscretization
Guider: VanillaCFG


Sampling with DPMPP2MSampler for 31 steps:  97%|█████████▋| 30/31 [00:03<00:00,  9.35it/s]


In [77]:
out = decode_samples(model, samples_rand)

In [78]:
perform_save_locally(SAVE_PATH, out)

In [14]:
potato_c = copy.copy(c)
potato_uc = copy.copy(uc)
potato_samples = copy.copy(samples)
potato_out = copy.copy(out)

In [21]:
cow_c = copy.copy(c)
cow_uc = copy.copy(uc)
cow_samples = copy.copy(samples)
cow_out = copy.copy(out)

In [36]:
c['crossattn'] = (potato_c['crossattn'] + cow_c['crossattn'])/2
c['vector'] = (potato_c['vector'] + cow_c['vector'])/2

In [44]:
samples = (potato_samples + cow_samples)/2

In [76]:
std = 1.0
mean = 10.0
random_sample = torch.randn(samples.shape).to('cuda') * std + mean
samples_rand = cow_samples + random_sample

In [48]:
print(c['crossattn'].mean(), c['vector'].mean())
print(samples.mean(), samples.std())
print(out.mean())

tensor(0.0208, device='cuda:0') tensor(0.1624, device='cuda:0')
tensor(0.0551, device='cuda:0') tensor(0.9009, device='cuda:0')
tensor(0.3431, device='cuda:0')


In [47]:
samples.shape

torch.Size([1, 4, 72, 128])

In [53]:
random_sample.mean()

tensor(0.0042, device='cuda:0')

In [80]:
out.shape

torch.Size([1, 3, 576, 1024])